In [12]:
import pandas
import pickle
import seaborn as sns
import platform
import re

In [5]:
YEWNO_DATA_PATH = '/Users/khoanguyen/Workspace/dataset/Yewno/ms_yewno_2020.pickle'
EDF_DATA_PATH = '/Users/khoanguyen/Workspace/dataset/edf_msft/MSFT_Jul2019_2020_linebreak'

# Windows path
if platform.system() == 'Windows':
    YEWNO_DATA_PATH = ''
    EDF_DATA_PATH = ''

In [3]:
with open(YEWNO_DATA_PATH, 'rb') as f:
    yewno_data = pickle.load(f)

In [6]:
with open(EDF_DATA_PATH, 'rb') as f:
    edf_data = pickle.load(f)

In [9]:
yewno_data['Contribution'].sum()

0.9771604537963867

In [13]:
concept_count = []
concept_list = yewno_data['Concept'].tolist()

for concept in concept_list:
    re_string = '\\b' + concept.lower() + '\\b'
    concept_per_doc = edf_data['text'].str.lower().apply(lambda x: len(re.findall(re_string, x)))
    concept_count.append(concept_per_doc.sum())

KeyboardInterrupt: 